## Step 0. Preparation

In [1]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [2]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'	     Vehicle_Tracking-master   yolov3_custom.cfg
 Instant-Noodles-Detection   yolov3		       yolov4


## Step 1. Cloning and Building Darknet

In [4]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 13728 (delta 2), reused 2 (delta 0), pack-reused 13714
Receiving objects: 100% (13728/13728), 12.37 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (9356/9356), done.


In [5]:
%cd darknet/
!rm src/image.c
!cp /mydrive/Instant-Noodles-Detection/image.c src/

/darknet


In [6]:
# change makefile to have GPU and OPENCV enabled
# %cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [7]:
# make darknet (build)
!make clean
!make

rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/rnn.o ./obj/rnn_vid.o ./obj/crnn_layer.o ./obj/demo.o ./obj/tag.o ./obj/cifar.o ./obj/go.o ./obj/batchnorm_layer.o ./obj/art.o ./obj/region_layer.o ./obj/reorg_layer.o ./obj/reorg_ol

## Step 2. Download pretrained YOLOv3 weights
YOLOv3 has been trained already on the coco dataset which has 80 classes that it can predict. 

In [ ]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

### Run Detections with Darknet and YOLOv3

In [ ]:
# run darknet detection
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg

In [ ]:
# show image using our helper function
imShow('predictions.jpg')

In [ ]:
# Stops 'Run all' at this cell by causing an error
assert False

## Step 3. Training Custom YOLOv3 Object Detector
In order to create a custom YOLOv3 detector we will need the following:

*   Labeled Custom Dataset
*   Custom .cfg file
*   obj.data and obj.names files
*   train.txt file (test.txt is optional here as well)

### Download pre-trained weights for the convolutional layers.
This step downloads the weights for the convolutional layers of the YOLOv3 network. By using these weights it helps your custom object detector to be way more accurate and not have to train as long. You don't have to use these weights but trust me it will help your modle converge and be accurate way faster.

In [10]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

--2020-06-21 08:43:30--  http://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/darknet53.conv.74 [following]
--2020-06-21 08:43:31--  https://pjreddie.com/media/files/darknet53.conv.74
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   633KB/s    in 3m 24s  

2020-06-21 08:46:56 (777 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



###  Moving Custom Dataset Into Your Cloud VM

In [ ]:
!cp -r /mydrive/Instant-Noodles-Detection/data/img data
!cp /mydrive/Instant-Noodles-Detection/data/train.txt data

In [8]:
# copy .data and .name
!cp /mydrive/Instant-Noodles-Detection/data/obj.data data
!cp /mydrive/Instant-Noodles-Detection/data/obj.names data

### Configuring Files for Training

In [11]:
!cp /mydrive/Instant-Noodles-Detection/yolov3_custom.cfg ./cfg

### Train

In [ ]:
# train your custom detector
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg /mydrive/Instant-Noodles-Detection/backup/yolov3_custom_last.weights darknet53.conv.74 -dont_show

In [ ]:
imShow('chart.png')

### Test

In [12]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

/darknet/cfg
/darknet


### picture

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /mydrive/Instant-Noodles-Detection/backup/yolov3_custom_last.weights /mydrive/Instant-Noodles-Detection/test.jpg -thresh 0.25 --dont-show
imShow("predictions.jpg")

In [ ]:
# !./darknet detector demo data/obj.data cfg/yolov3_custom.cfg /mydrive/Instant-Noodles-Detection/backup/yolov3_custom_last_0619.weights /mydrive/Instant-Noodles-Detection/supermarket.mp4 -out_filename /mydrive/Instant-Noodles-Detection/result.avi

In [ ]:
# test all frame
import os
files = os.listdir("/mydrive/Instant-Noodles-Detection/test/")
files.sort(key=lambda x: int(x.replace("frame","").split('.')[0]))
for filename in files:
  print(filename)
  !./darknet detector test data/obj.data cfg/yolov3_custom.cfg /mydrive/Instant-Noodles-Detection/backup/yolov3_custom_last.weights /mydrive/Instant-Noodles-Detection/test/$filename -thresh 0.3
  download('predictions.jpg')

In [ ]:
# test significant part
import os
files = os.listdir("/mydrive/Instant-Noodles-Detection/test/")
files.sort(key=lambda x: int(x.replace("frame","").split('.')[0]))
for filename in files:
  print(filename)
  index = int(filename[filename.find('e')+1:filename.find('.')])
  if index > 200 and index < 330:
    !./darknet detector test data/obj.data cfg/yolov3_custom.cfg /mydrive/Instant-Noodles-Detection/backup/yolov3_custom_last.weights /mydrive/Instant-Noodles-Detection/test/$filename -thresh 0.3
    download('predictions.jpg')

### video

In [18]:
!./darknet detector demo data/obj.data cfg/yolov3_custom.cfg /mydrive/Instant-Noodles-Detection/backup/yolov3_custom_0619.weights -dont_show /mydrive/Instant-Noodles-Detection/supermarket.mp4 -i 0 -out_filename results.avi -thresh 0.28

流式输出内容被截断，只能显示最后 5000 行内容。

 cvWriteFrame 
Objects:

luxiang: 95% 
luxiang: 94% 
xiangla: 99% 
xiangla: 90% 
xiangla: 85% 
xiangla: 81% 
xiangla: 66% 

FPS:14.7 	 AVG_FPS:13.9

 cvWriteFrame 
Objects:

luxiang: 97% 
luxiang: 94% 
xiangla: 99% 
xiangla: 93% 
xiangla: 93% 
xiangla: 86% 
xiangla: 75% 
xiangla: 61% 

FPS:14.8 	 AVG_FPS:13.9

 cvWriteFrame 
Objects:

luxiang: 97% 
luxiang: 92% 
xiangla: 99% 
xiangla: 97% 
xiangla: 93% 
xiangla: 91% 
xiangla: 78% 

FPS:14.8 	 AVG_FPS:13.9

 cvWriteFrame 
Objects:

luxiang: 94% 
luxiang: 76% 
luxiang: 47% 
xiangla: 98% 
xiangla: 96% 
xiangla: 93% 
xiangla: 92% 
xiangla: 57% 

FPS:14.9 	 AVG_FPS:13.9

 cvWriteFrame 
Objects:

luxiang: 93% 
luxiang: 54% 
luxiang: 47% 
xiangla: 99% 
xiangla: 91% 
xiangla: 89% 
xiangla: 71% 
xiangla: 53% 

FPS:14.9 	 AVG_FPS:13.9

 cvWriteFrame 
Objects:

luxiang: 98% 
luxiang: 46% 
luxiang: 42% 
luxiang: 33% 
xiangla: 98% 
xiangla: 98% 
xiangla: 90% 
xiangla: 69% 
xiangla: 46% 

FPS:14.9 	 AVG_FPS:13.9

 cvWrite

In [20]:
download('results.avi')